In [13]:
from kubernetes import client, config,utils
import pprint

# Kubernete Api Client


In [5]:
config.load_kube_config()
k8s_client = client.ApiClient() 
yaml_file = './vcjob.yaml'
utils.create_from_yaml(k8s_client,yaml_file,verbose=True)

AttributeError: module 'kubernetes.client' has no attribute 'BatchVolcanoShV1alpha1Api'

# Kubernetes CRD Client

In [14]:
volcano_job_body = {
    "apiVersion": "batch.volcano.sh/v1alpha1",
    "kind": "Job",
    "metadata": {
        "name": "test-job"
    },
    "spec": {
        "schedulerName": "volcano",
        "priorityClassName": "h-priority",
        "policies": [
            {
                "event": "PodEvicted",
                "action": "RestartJob"
            }
        ],
        "maxRetry": 5,
        "tasks": [
            {
                "name": "default-nginx",
                "template": {
                    "metadata": {
                        "name": "web"
                    },
                    "spec": {
                        "containers": [
                            {
                                "image": "nginx",
                                "imagePullPolicy": "IfNotPresent",
                                "name": "nginx",
                                "resources": {
                                    "requests": {
                                        "cpu": "1"
                                    }
                                }
                            }
                        ]
                    }
                }
            }
        ]
    }
}

# CRD Create

In [8]:
config.load_kube_config()
api = client.CustomObjectsApi()

# CRD Get

In [9]:
resource = api.get_namespaced_custom_object(
    group="scheduling.volcano.sh",
    version="v1beta1",
    name="podgroup-e029423c-dc67-45b9-9f80-ebd941fc1605",
    namespace="argo",
    plural="podgroups"
)

In [10]:
import pprint 
pprint.pprint(resource)

{'apiVersion': 'scheduling.volcano.sh/v1beta1',
 'kind': 'PodGroup',
 'metadata': {'creationTimestamp': '2023-05-30T10:56:45Z',
              'generation': 2015,
              'managedFields': [{'apiVersion': 'scheduling.volcano.sh/v1beta1',
                                 'fieldsType': 'FieldsV1',
                                 'fieldsV1': {'f:metadata': {'f:ownerReferences': {'.': {},
                                                                                   'k:{"uid":"e029423c-dc67-45b9-9f80-ebd941fc1605"}': {'.': {},
                                                                                                                                        'f:apiVersion': {},
                                                                                                                                        'f:blockOwnerDeletion': {},
                                                                                                                                        'f:con

# CRD Information

In [11]:
status = resource['status']['phase'] # 상태
priorityClassName = resource['spec']['priorityClassName'] # penalty value
queue = resource['spec']['queue'] # queue

# Volcano 스케줄러 사이클

1. Kubeflow에서 pipeline을 돌림  
2. Argo workflow 형태로 작업이 시작됨  
    - workflow 형태는 시작되면 Active 완료시 Succeeded 상태가 됨  
3. schedulerName과 podPriorityClassName, queue 정보 등을 기입함  
4. 생성된 workflow는 지정한 queue에 podgroup의 crd 형태로 시작된다.  
5. 완료된 podgroup은 삭제되지 않는다.
    - podgroup은 namespace, queue별로 지정되서 들어간다.
    - 이걸 활용해서 penalty를 계산해야하나?  
6. 패널티 계산 로직에 따라 패널티를 계산하고 완화한다.
    - 패널티는 priorityclass의 value를 통해 계산한다.

# PriorityClassName GET

In [12]:
priorityClassResource = api.get_cluster_custom_object(
    group="scheduling.k8s.io",
    version="v1",
    name="h-priority",
    plural="priorityclasses"
)

In [14]:
pprint.pprint(priorityClassResource)

{'apiVersion': 'scheduling.k8s.io/v1',
 'kind': 'PriorityClass',
 'metadata': {'creationTimestamp': '2023-05-26T06:44:33Z',
              'generation': 1,
              'managedFields': [{'apiVersion': 'scheduling.k8s.io/v1',
                                 'fieldsType': 'FieldsV1',
                                 'fieldsV1': {'f:preemptionPolicy': {},
                                              'f:value': {}},
                                 'manager': 'OpenAPI-Generator',
                                 'operation': 'Update',
                                 'time': '2023-05-26T06:44:33Z'}],
              'name': 'h-priority',
              'resourceVersion': '15476',
              'uid': 'bfbe9c75-151e-43e7-90b8-2d23fca8ce9a'},
 'preemptionPolicy': 'PreemptLowerPriority',
 'value': 1000}


In [18]:
penaltyValue = priorityClassResource['value']

penaltyValue

1000

In [17]:
value_patch = {
    "value": "1001"
}

In [19]:
priority_body = {
    "apiVersion": "scheduling.k8s.io/v1",
    "kind": "PriorityClass",
    "metadata": {"name" : "high-priority"},
    "preemptionPolicy": "PreemptLowerPriority",
    "value" : "10001"
}

In [26]:
created_resource = api.create_cluster_custom_object(
    group="scheduling.k8s.io",
    version="v1",
    plural="priorityclasses",
    body=priority_body
)

ApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Cache-Control': 'no-cache, private', 'Content-Type': 'application/json', 'X-Kubernetes-Pf-Flowschema-Uid': 'd7846aa7-2000-49fb-81f3-7f033dd11d7e', 'X-Kubernetes-Pf-Prioritylevel-Uid': '99e29715-90d7-43c1-b132-98f418952660', 'Date': 'Thu, 01 Jun 2023 04:05:57 GMT', 'Content-Length': '396'})
HTTP response body: {"kind":"Status","apiVersion":"v1","metadata":{},"status":"Failure","message":"PriorityClass in version \"v1\" cannot be handled as a PriorityClass: v1.PriorityClass.Value: readUint32: unexpected character: \ufffd, error found in #10 byte of ...|\"value\": \"10001\"}|..., bigger context ...|emptionPolicy\": \"PreemptLowerPriority\", \"value\": \"10001\"}|...","reason":"BadRequest","code":400}



In [20]:
patched_resource = api.patch_cluster_custom_object(
    group="scheduling.k8s.io",
    version="v1",
    plural="priorityclasses",
    name="h-priority",
    body=value_patch
)

ApiException: (422)
Reason: Unprocessable Entity
HTTP response headers: HTTPHeaderDict({'Cache-Control': 'no-cache, private', 'Content-Type': 'application/json', 'X-Kubernetes-Pf-Flowschema-Uid': 'd7846aa7-2000-49fb-81f3-7f033dd11d7e', 'X-Kubernetes-Pf-Prioritylevel-Uid': '99e29715-90d7-43c1-b132-98f418952660', 'Date': 'Thu, 01 Jun 2023 01:19:54 GMT', 'Content-Length': '2044'})
HTTP response body: {"kind":"Status","apiVersion":"v1","metadata":{},"status":"Failure","message":" \"\" is invalid: patch: Invalid value: \"{\\\"apiVersion\\\":\\\"scheduling.k8s.io/v1\\\",\\\"kind\\\":\\\"PriorityClass\\\",\\\"metadata\\\":{\\\"name\\\":\\\"h-priority\\\",\\\"uid\\\":\\\"bfbe9c75-151e-43e7-90b8-2d23fca8ce9a\\\",\\\"resourceVersion\\\":\\\"15476\\\",\\\"generation\\\":1,\\\"creationTimestamp\\\":\\\"2023-05-26T06:44:33Z\\\",\\\"managedFields\\\":[{\\\"manager\\\":\\\"OpenAPI-Generator\\\",\\\"operation\\\":\\\"Update\\\",\\\"apiVersion\\\":\\\"scheduling.k8s.io/v1\\\",\\\"time\\\":\\\"2023-05-2

In [21]:
test_resource = {
        "apiVersion": "stable.example.com/v1",
        "kind": "CronTab",
        "metadata": {"name": "test-crontab"},
        "spec": {"cronSpec": "* * * * */5", "image": "my-awesome-cron-image"},
    }

cronspec_patch = {
        "spec": {"cronSpec": "* * * * */15", "image": "my-awesome-cron-image"}
    }

# patch to add the `metadata.labels` field
metadata_label_patch = {
    "metadata": {
        "labels": {
            "foo": "bar",
        }
    }
}

created_resource = api.create_cluster_custom_object(
    group="stable.example.com",
    version="v1",
    plural="crontabs",
    body=test_resource,     
)

ApiException: (404)
Reason: Not Found
HTTP response headers: HTTPHeaderDict({'Cache-Control': 'no-cache, private', 'Content-Type': 'text/plain; charset=utf-8', 'X-Content-Type-Options': 'nosniff', 'X-Kubernetes-Pf-Flowschema-Uid': 'd7846aa7-2000-49fb-81f3-7f033dd11d7e', 'X-Kubernetes-Pf-Prioritylevel-Uid': '99e29715-90d7-43c1-b132-98f418952660', 'Date': 'Thu, 01 Jun 2023 01:20:08 GMT', 'Content-Length': '19'})
HTTP response body: 404 page not found



In [23]:
v1 = client.CoreV1Api()
v1.

{'api_version': 'v1',
 'items': [{'api_version': None,
            'kind': None,
            'metadata': {'annotations': None,
                         'cluster_name': None,
                         'creation_timestamp': datetime.datetime(2023, 5, 28, 7, 55, 20, tzinfo=tzutc()),
                         'deletion_grace_period_seconds': None,
                         'deletion_timestamp': None,
                         'finalizers': None,
                         'generate_name': None,
                         'generation': None,
                         'labels': {'kubernetes.io/metadata.name': 'argo'},
                         'managed_fields': [{'api_version': 'v1',
                                             'fields_type': 'FieldsV1',
                                             'fields_v1': {'f:metadata': {'f:labels': {'.': {},
                                                                                       'f:kubernetes.io/metadata.name': {}}}},
                            